In [1]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from importlib import reload 
import sys 
import warnings
import simfin
warnings.filterwarnings('ignore')

Afin de faire des réplications, on doit définir une fonction englobant la simulation et lui appliquer le décorateur `@simulator_instance.replication(rep=1)`.

In [2]:
x = simfin.simulator(2019,2040)

@x.replication(rep=1)
def premiere_simulatation(self):    #L'argument self est nécessaire pour permettre l'accès aux éléments de la simulation
    self.simulate() 

alignment factor for health :  1.2485447430148122
alignment factor for education :  1.4417133670200024


In [3]:
%%time
premiere_simulatation()

CPU times: user 1min 19s, sys: 1.25 s, total: 1min 20s
Wall time: 1min 31s


L'ensemble des `summary` sont dans une liste `x.summary`. On peut facilement les regrouper et en obtenir la moyenne, l'écart type ou encore, le 5ième et 95ième percentille. Cependant, dans cette exemple simple il n'y a pas de variation entre les réplications et l'ensemble des réplications sont identiques.

In [81]:
tmp = pd.concat(x.all_summary).astype(float)
tmp.groupby(level=0).mean().head(5)

,2019,2020,2021,2022,2023,2024,2025,2026,2027,2028,...,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039
autonomous,73505.0,93560.326667,95538.430200,97561.350159,99630.151566,101745.925543,103909.789960,106122.890111,108386.399391,110701.520004,...,115491.552432,117969.019270,120503.209027,123095.479125,125747.220403,128459.857952,131234.851972,134073.698654,136977.931083,139949.120160
consumption,21001.0,21444.354444,21897.742700,22361.403037,22835.579563,23320.522366,23816.487667,24323.737970,24842.542222,25373.175972,...,26471.068175,27038.912249,27619.757413,28213.914796,28821.703181,29443.449205,30079.487550,30730.161149,31395.821390,32076.828329
corporate,9183.0,9376.863333,9575.114100,9777.856487,9985.197235,10197.245697,10414.113911,10635.916660,10862.771545,11094.799055,...,11574.868771,11823.167048,12077.150247,12336.954410,12602.718933,12874.586641,13152.703879,13437.220601,13728.290454,14026.070880
debt,189044.0,190335.198075,191682.278078,192624.689987,190716.116518,188437.488109,159580.743716,152661.083964,146425.744233,139936.329872,...,126870.021058,119866.566613,112519.903353,105170.069598,98058.088377,90328.035053,80718.996829,69600.410201,57750.851970,45209.124941
debt service,8722.0,7884.485114,7938.337297,7994.520155,8033.825463,7954.224317,7859.189236,6655.656875,6367.057638,6106.999433,...,5568.217836,5291.386093,4999.292018,4692.883683,4386.343117,4089.722700,3767.324262,3366.558732,2902.834251,2408.623033


Nous pouvons définir des fonctions qui effectuent des modifications avant chaque réplication. Encore une fois, la fonction doit avoir comme argument `self` et un autre argument qui peut prendre la forme d'un scalaire ou tout autre forme.

In [57]:
def change_inflation(self,inf):
    self.macro.infl = inf
    #print(self.macro.infl) #on peut ici vérifier que l'on modifie la valeur à chaque réplication


Le décorateur `replication` peut prendre comme deuxième argument `param` un dictionaire qui à comme clé(s) une ou des fonctions à appliquer avant chaque réplication ainsi qu'une liste ou un tuple qui comprend en position 0 une liste de valeur et en position 1 une liste de même longeur comprenant la distribution cummulative des probabilités pour chaqu'une des valeurs en 0. À chaque réplication une valeur entre 0 et 1 est tiré automatiquement et la valeur en 0 correspondante est passé à la fonction qui sert de clé.

In [72]:
infl = np.linspace(0,.03,10) #valeur possible pour l'inflation (entre 0% et 3%)
prob = np.linspace(0.1,1,10) #La distribution cummulative des valeurs précédentes.
params = [infl,prob] #La liste

In [70]:
@x.replication(rep=100,param={change_inflation:params})
def deuxieme_simulation(self):
    self.simulate()

In [71]:
%%time
deuxieme_simulation()

CPU times: user 1min 22s, sys: 3.22 s, total: 1min 25s
Wall time: 1min 36s


In [73]:
tmp = pd.concat(x.all_summary).astype(float)
tmp.groupby(level=0).mean().head(5)

,2019,2020,2021,2022,2023,2024,2025,2026,2027,2028,...,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039
autonomous,73505.0,92887.384600,94174.353215,95487.494458,96827.411497,98194.722433,99590.060687,101014.075396,102467.431820,103950.811763,...,107010.454722,108588.167979,110198.806159,111843.140454,113521.961359,115236.079166,116986.324491,118773.548796,120598.624940,122462.447737
consumption,21001.0,21290.113767,21585.091479,21886.068050,22193.181726,22506.574180,22826.390593,23152.779750,23485.894131,23825.890008,...,24527.170886,24888.788289,25257.952199,25634.839376,26019.631005,26412.512808,26813.675165,27223.313233,27641.627075,28068.821786
corporate,9183.0,9309.419300,9438.402698,9570.009185,9704.299214,9841.334731,9981.179221,10123.897740,10269.556964,10418.225225,...,10724.870732,10882.993327,11044.415744,11209.215275,11377.471145,11549.264564,11724.678779,11903.799125,12086.713082,12273.510331
debt,189044.0,190326.098309,191654.340065,192749.617750,192415.218814,191770.850161,164436.978883,161020.960726,157608.022526,153553.972803,...,143755.008281,137593.627286,130795.716918,122977.641124,114260.130259,104720.108574,93835.507644,81798.076630,68886.915213,55287.033064
debt service,8722.0,7884.485114,7937.957772,7993.354940,8039.035843,8025.089019,7998.214243,6858.196569,6715.724212,6573.380311,...,6217.052720,5995.610667,5738.637069,5455.115651,5129.046047,4765.463576,4367.576528,3913.610922,3411.564067,2873.076414


In [74]:
tmp.groupby(level=0).std().head(5)

,2019,2020,2021,2022,2023,2024,2025,2026,2027,2028,...,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039
autonomous,0.0,872.167786,1769.438164,2692.545375,3642.245216,4619.315679,5624.557593,6658.795298,7722.877330,8817.677132,...,11103.052736,12295.506619,13522.436000,14784.850219,16083.788229,17420.319462,18795.544717,20210.597086,21666.642893,23164.882666
consumption,0.0,199.903910,405.561422,617.140827,834.815356,1058.763327,1289.168293,1526.219196,1770.110523,2021.042471,...,2544.858561,2818.173166,3099.389676,3388.739435,3686.460575,3992.798212,4308.004656,4632.339613,4966.070410,5309.472212
corporate,0.0,87.410962,177.337772,269.854017,365.035447,462.960032,563.708035,667.362072,774.007187,883.730918,...,1112.777304,1232.288185,1355.254292,1481.776783,1611.959786,1745.910480,1883.739191,2025.559481,2171.488242,2321.645794
debt,0.0,11.793769,36.229644,569.439623,2800.017229,5295.704498,7785.825818,12418.475906,16843.965923,21454.296305,...,31370.109241,36559.481198,42241.500952,47683.188272,53100.424581,59419.701088,65321.133260,71632.830120,78682.437903,86519.145929
debt service,0.0,0.000000,0.491884,1.511035,23.749700,116.780719,220.868704,324.724550,517.939149,702.513693,...,1098.481687,1308.357628,1524.791505,1761.772315,1988.729545,2214.666994,2478.225962,2724.357836,2987.600679,3281.619678


Il est bien sur possible de définir des simulations plus complexe en changeant des éléments dans la fonction de simulation.

In [83]:
@x.replication(rep=100,param={change_inflation:params})
def troisième_simulation(self):
    self.simulate(1)
    #Intervertation/changement défini (p.e. taux de chômage à 17% comme dans l'EPA d'avril) 
    self.simulate()

In [84]:
%%time
troisième_simulation()

CPU times: user 1min 23s, sys: 3.22 s, total: 1min 26s
Wall time: 1min 41s


In [85]:
tmp = pd.concat(x.all_summary).astype(float)
tmp.groupby(level=0).mean().head(5)

,2019,2020,2021,2022,2023,2024,2025,2026,2027,2028,...,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039
autonomous,73505.0,93009.552600,94423.250154,95867.822514,97344.018286,98852.605329,100394.371270,101970.124030,103580.692367,105226.926432,...,108629.902768,110388.457534,112186.304244,114024.408917,115903.762638,117825.382237,119790.310974,121799.619254,123854.405352,125955.796169
consumption,21001.0,21318.115100,21642.139529,21973.240572,22311.589811,22657.363243,23010.741395,23371.909444,23741.057346,24118.379958,...,24898.354048,25301.420958,25713.493718,26134.793745,26565.548198,27005.990138,27456.358684,27916.899176,28387.863345,28869.509477
corporate,9183.0,9321.663300,9463.347807,9608.126669,9756.074912,9907.269495,10061.789354,10219.715462,10381.130880,10546.120811,...,10887.176098,11063.423106,11243.608057,11427.827768,11616.181568,11808.771365,12005.701719,12207.079908,12413.016004,12623.622948
debt,189044.0,190327.750309,191659.429032,192640.521242,191905.023183,190820.501316,163032.397818,158814.067383,154528.742939,149552.581224,...,137836.144594,130810.868810,123143.494769,114441.021733,104869.107777,94398.221387,82769.952955,70012.237220,56343.125526,41919.704860
debt service,8722.0,7884.485114,7938.026672,7993.567187,8034.485740,8003.810217,7958.577908,6799.615506,6623.680996,6444.952357,...,6009.246451,5748.751773,5455.747593,5135.963328,4773.008042,4373.790859,3937.080105,3452.098252,2920.010379,2349.910785


In [86]:
tmp.groupby(level=0).std().head(5)

,2019,2020,2021,2022,2023,2024,2025,2026,2027,2028,...,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039
autonomous,0.0,925.423058,1878.332005,2859.552198,3869.933675,4910.351881,5981.708427,7084.931854,8220.978441,9390.833020,...,11836.053350,13113.539269,14429.075333,15783.802332,17178.895069,18615.563368,20095.053108,21618.647290,23187.667135,24803.473213
consumption,0.0,212.110205,430.520272,655.419376,887.002348,1125.469843,1371.028514,1623.891187,1884.277042,2152.411807,...,2712.864868,3005.669114,3307.194585,3617.702757,3937.462896,4266.752300,4605.856529,4955.069650,5314.694492,5685.042902
corporate,0.0,92.748346,188.251400,286.591883,387.854986,492.128450,599.502636,710.070605,823.928197,941.174117,...,1186.240564,1314.273581,1446.120084,1581.894406,1721.714289,1865.700984,2013.979358,2166.677996,2323.929314,2485.869671
debt,0.0,12.513906,38.453357,664.280753,3085.960332,5786.714584,8533.278103,13473.733046,18180.295744,23132.906403,...,33921.836006,39728.217669,46062.498631,52069.534835,57959.543959,64743.858749,71250.397845,78266.194429,86041.617855,94639.148440
debt service,0.0,0.000000,0.521919,1.603780,27.705252,128.706588,241.347332,355.898649,561.950909,758.248192,...,1184.000038,1414.782860,1656.950450,1921.135211,2171.671528,2417.326980,2700.281366,2971.650522,3264.259352,3588.550048
